In [ ]:
import pandas as pd

# Define the normalization function
def normalize(df, column_name, high_good=True):
    if high_good:
        df[column_name + ' Score'] = ((df[column_name] - df[column_name].min()) /
                                      (df[column_name].max() - df[column_name].min()) * 100)
    else:
        df[column_name + ' Score'] = ((df[column_name].max() - df[column_name]) /
                                      (df[column_name].max() - df[column_name].min()) * 100)


In [ ]:
# Hypothetical data for the banks
banks_data = {
    'Bank': ['Bank of Palestine', 'Palestine Islamic Bank', 'The National Bank (TNB)', 'Arab Islamic Bank',
             'Cairo Amman Bank', 'Palestinian Investment Bank', 'Al Quds Bank', 'Bank of Jordan',
             'Safa Bank', 'Islamic International Arab Bank', 'Jordan Kuwait Bank', 'Palestine Commercial Bank'],
    'Interest Rate': [5.0, 6.5, 4.5, 7.0, 4.2, 5.6, 7.8, 10.2, 5.3, 7.2, 6.3, 6.2],
    'Loan Term': [80, 90, 85, 70, 60, 80, 70, 97, 65, 88, 90, 70],
    'Customer Service Rating': [4.5, 3.0, 4.0, 4.8, 3.2, 4.2, 4.8, 3.0, 4.2, 4.1, 3.7, 3.9],
    'Fees and Charges': [500, 800, 450, 900, 500, 700, 900, 450, 660, 450, 630, 540],
    'Loan Amount Limits': [50000, 75000, 60000, 80000, 60000, 70000, 86000, 67000, 100000, 45000, 80000, 70000],
    'Speed of Approval (days)': [2, 3, 1, 5, 4, 5, 2, 1, 1, 4, 2, 3],
    'Eligibility Requirements Score': [70, 60, 80, 50, 50, 50, 60, 60, 70, 50, 60, 80],
    'Flexibility Score': [75, 65, 85, 60, 60, 76, 80, 50, 70, 60, 80, 86]
    'ATM amount':

}


In [ ]:
# Convert the dictionary to a DataFrame
banks_df = pd.DataFrame(banks_data)

# Normalize the criteria in the banks DataFrame
normalize(banks_df, 'Interest Rate', high_good=False)
normalize(banks_df, 'Fees and Charges', high_good=False)
normalize(banks_df, 'Loan Amount Limits', high_good=True)
normalize(banks_df, 'Speed of Approval (days)', high_good=False)
banks_df['Customer Service Rating Score'] = banks_df['Customer Service Rating'] / 5 * 100

In [ ]:
# Filter banks based on user preferences


# User preferences
user_preferences = {
    'max_interest_rate':6.0,  # The maximum interest rate the user is willing to accept
    'min_loan_term': 70,       # The minimum loan term in months the user needs
    'min_loan_amount': 10000,  # The minimum loan amount the user needs
    'max_fees': 900,           # The maximum fees the user is willing to pay
    'max_approval_days':3,    # The maximum number of days the user is willing to wait for approval
}

# Define weights for each criterion
weights = {
    'Interest Rate Score': 0.40,
    'Loan Term': 0.20,
    'Fees and Charges Score': 0.10,
    'Loan Amount Limits Score': 0.20,
    'Speed of Approval (days) Score': 0.10,
}

# Convert to DataFrame for easier manipulation

# Filtering banks based on user preferences
filtered_banks = banks_df[
    (banks_df['Interest Rate'] <= user_preferences['max_interest_rate']) &
    (banks_df['Loan Term']  >= user_preferences['min_loan_term']) &  # Assuming Loan Term Score is in years
    (banks_df['Loan Amount Limits'] >= user_preferences['min_loan_amount']) &
    (banks_df['Fees and Charges'] <= user_preferences['max_fees']) &
    (banks_df['Speed of Approval (days)'] <= user_preferences['max_approval_days'])
]

# Check the filtered banks
filtered_banks


,Bank,Interest Rate,Loan Term,Customer Service Rating,Fees and Charges,Loan Amount Limits,Speed of Approval (days),Eligibility Requirements Score,Flexibility Score,Interest Rate Score,Fees and Charges Score,Loan Amount Limits Score,Speed of Approval (days) Score,Customer Service Rating Score
0,Bank of Palestine,5.0,80,4.5,500,50000,2,70,75,86.666667,88.888889,9.090909,75.0,90.0
2,The National Bank (TNB),4.5,85,4.0,450,60000,1,80,85,95.000000,100.000000,27.272727,100.0,80.0


In [ ]:
# Create a copy of the original DataFrame to avoid modifying it directly
df_banks_copy = banks_df.copy()
def score(value, ideal, is_lower_better):
    """
    Scores a value based on how close it is to the ideal value.
    :param value: The actual value.
    :param ideal: The ideal value.
    :param is_lower_better: True if lower values are better, False otherwise.
    :return: Score between 0 and 1.
    """
    if is_lower_better:
        return max(0, min(1, ideal / value))
    else:
        return max(0, min(1, value / ideal))
# Reapply the scoring on the copied DataFrame
for index, row in df_banks_copy.iterrows():
    df_banks_copy.loc[index, 'Interest Rate Score'] = score(row['Interest Rate'], user_preferences['max_interest_rate'], True)
    df_banks_copy.loc[index, 'Loan Term'] = score(row['Loan Term'], user_preferences['min_loan_term'], False)
    df_banks_copy.loc[index, 'Loan Amount Limits Score'] = score(row['Loan Amount Limits'], user_preferences['min_loan_amount'], False)
    df_banks_copy.loc[index, 'Fees and Charges Score'] = score(row['Fees and Charges'], user_preferences['max_fees'], True)
    df_banks_copy.loc[index, 'Speed of Approval (days) Score'] = score(row['Speed of Approval (days)'], user_preferences['max_approval_days'], True)

# Calculate total score for each bank in the copied DataFrame
banks_df['Total Score'] = (
    df_banks_copy['Interest Rate Score'] * weights['Interest Rate Score'] +
    df_banks_copy['Loan Term'] * weights['Loan Term'] +
    df_banks_copy['Loan Amount Limits Score'] * weights['Loan Amount Limits Score'] +
    df_banks_copy['Fees and Charges Score'] * weights['Fees and Charges Score'] +
    df_banks_copy['Speed of Approval (days) Score'] * weights['Speed of Approval (days) Score']
)

# Sort banks by total score in descending order in the copied DataFrame
sorted_banks_copy = banks_df.sort_values(by='Total Score', ascending=False)

# Display the top ranked banks from the copied DataFrame
sorted_banks_copy[['Bank', 'Total Score', 'Interest Rate', 'Loan Term', 'Loan Amount Limits', 'Fees and Charges', 'Speed of Approval (days)']]


,Bank,Total Score,Interest Rate,Loan Term,Loan Amount Limits,Fees and Charges,Speed of Approval (days)
0,Bank of Palestine,1.000000,5.0,80,50000,500,2
2,The National Bank (TNB),1.000000,4.5,85,60000,450,1
11,Palestine Commercial Bank,0.987097,6.2,70,70000,540,3
8,Safa Bank,0.985714,5.3,65,100000,660,1
10,Jordan Kuwait Bank,0.980952,6.3,90,80000,630,2
1,Palestine Islamic Bank,0.969231,6.5,90,75000,800,3
5,Palestinian Investment Bank,0.960000,5.6,80,70000,700,5
4,Cairo Amman Bank,0.946429,4.2,60,60000,500,4
9,Islamic International Arab Bank,0.908333,7.2,88,45000,450,4
6,Al Quds Bank,0.907692,7.8,70,86000,900,2


In [ ]:
from sklearn.cluster import KMeans
import numpy as np


clustering_features = ['Interest Rate', 'Loan Term', 'Fees and Charges', 'Loan Amount Limits', 'Speed of Approval (days)']
clustering_data = banks_df[clustering_features]


kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=500, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(clustering_data)

banks_df['Cluster'] = cluster_labels

user_prefs = [
    '3',  # The maximum interest rate the user is willing to accept
    '60',       # The minimum loan term in months the user needs
    '10000',  # The minimum loan amount the user needs
    '900',           # The maximum fees the user is willing to pay
    '5' # The maximum number of days the user is willing to wait for approval

  ]  # These should ideally be normalized as well

# Finding the closest cluster to the user's preferences
# We calculate the Euclidean distance from the user's preferences to each cluster centroid
centroids = kmeans.cluster_centers_
user_prefs = np.array(user_prefs).astype(float)  # Convert user_prefs to float

distances = [np.linalg.norm(centroid - user_prefs) for centroid in centroids]
closest_cluster = np.argmin(distances)

# Filtering banks from the closest cluster
recommended_banks = banks_df[banks_df['Cluster'] == closest_cluster]

recommended_banks[['Bank', 'Cluster']]  # Displaying banks from the closest cluster


,Bank,Cluster
0,Bank of Palestine,2
2,The National Bank (TNB),2
4,Cairo Amman Bank,2
9,Islamic International Arab Bank,2


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture


agglo_cluster_model = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
agglo_clusters = agglo_cluster_model.fit_predict(clustering_data)



gmm_model = GaussianMixture(n_components=4, random_state=0)
gmm_model.fit(clustering_data)
gmm_clusters = gmm_model.predict(clustering_data)

banks_df['GMM Cluster'] = gmm_clusters
banks_df['Agglo Cluster'] = agglo_clusters


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [ ]:
## User preferences for demonstration
# Hypothetical user preferences

# For GMM: Find the closest cluster by calculating the distance to each GMM cluster centroid
gmm_centroids = gmm_model.means_
gmm_distances = [np.linalg.norm(centroid - user_prefs) for centroid in gmm_centroids]
closest_gmm_cluster = np.argmin(gmm_distances)

# For Agglomerative Clustering: Calculate the mean of features for each cluster and find the closest one
agglo_centroids = banks_df.groupby('Agglo Cluster')[clustering_features].mean().values
agglo_distances = [np.linalg.norm(centroid - user_prefs) for centroid in agglo_centroids]
closest_agglo_cluster = np.argmin(agglo_distances)

banks_from_gmm_cluster = banks_df[banks_df['GMM Cluster'] == closest_gmm_cluster]
banks_from_agglo_cluster = banks_df[banks_df['Agglo Cluster'] == closest_agglo_cluster]

# Displaying the banks from each cluster
banks_from_gmm_cluster_list = banks_from_gmm_cluster['Bank'].tolist()
banks_from_agglo_cluster_list = banks_from_agglo_cluster['Bank'].tolist()

banks_from_gmm_cluster_list, banks_from_agglo_cluster_list




(['Bank of Palestine', 'Islamic International Arab Bank'],
 ['Bank of Palestine', 'Islamic International Arab Bank'])

In [ ]:
# Assuming the existing code up to 'banks_from_gmm_cluster_list, banks_from_agglo_cluster_list'

# Find the common banks in both lists
common_banks = set(banks_from_gmm_cluster_list).intersection(banks_from_agglo_cluster_list)

# Define a method to select the best bank
def select_best_bank():
    if common_banks:
        # If there are common banks, choose one as the best recommendation
        return common_banks.pop()
    elif banks_from_gmm_cluster_list and banks_from_agglo_cluster_list:
        # If no common banks, choose the first bank from the GMM list (or any other criteria)
        return banks_from_gmm_cluster_list[0]
    else:
        # No banks to recommend
        return "No recommended banks available."

# Print the best recommended bank
best_bank = select_best_bank()
print("Best Recommended Bank:", best_bank)


Best Recommended Bank: Bank of Palestine
